można puścić całe od początku

In [20]:
import numpy as np
import tools
import gym
import matplotlib.pyplot as plt

env = gym.make('FrozenLake8x8-v1')
gamma = 0.99
theta = 0.000001
# https://github.com/SaiSugunSegu/Reinforcement-Learning-Journey

In [21]:
env.action_space #env.action_space

Discrete(4)

In [22]:
env.observation_space

Discrete(64)

In [23]:
env.action_space.n #env.env.nA

4

In [24]:
env.observation_space.n #env.env.nS

64

In [25]:
env.unwrapped.P[0] #[1] #anv.env.P[0][1] [pozycja_na_planszy][wykonywany ruch]

{0: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 8, 0.0, False)],
 1: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 8, 0.0, False),
  (0.3333333333333333, 1, 0.0, False)],
 2: [(0.3333333333333333, 8, 0.0, False),
  (0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)]}

In [26]:
def argmax(env, V, pi, action,s, gamma):
    e = np.zeros(env.action_space.n)
    for a in range(env.action_space.n):                         # iterate for every action possible 
        q=0
        P = np.array(env.unwrapped.P[s][a])                   
        (x,y) = np.shape(P)                             # for Bellman Equation 
        
        for i in range(x):                              # iterate for every possible states
            s_= int(P[i][1])                            # S' - Sprime - possible succesor states
            p = P[i][0]                                 # Transition Probability P(s'|s,a) 
            r = P[i][2]                                 # Reward
            
            q += p*(r+gamma*V[s_])                      # calculate action_ value q(s|a)
            e[a] = q
            
    m = np.argmax(e) 
    action[s]=m                                           # Take index which has maximum value 
    pi[s][m] = 1                                        # update pi(a|s) 

    return pi

In [27]:
def bellman_optimality_update(env, V, s, gamma):  # update the stae_value V[s] by taking 
    pi = np.zeros((env.observation_space.n, env.action_space.n))       # action which maximizes current value
    e = np.zeros(env.action_space.n)                       
                                            # STEP1: Find 
    for a in range(env.action_space.n):             
        q=0                                 # iterate for all possible action
        P = np.array(env.unwrapped.P[s][a])
        (x,y) = np.shape(P)
        
        for i in range(x):
            s_= int(P[i][1])
            p = P[i][0]
            r = P[i][2]
            q += p*(r+gamma*V[s_])
            e[a] = q
            
    m = np.argmax(e)
    pi[s][m] = 1
    
    value = 0
    for a in range(env.action_space.n):
        u = 0
        P = np.array(env.unwrapped.P[s][a])
        (x,y) = np.shape(P)
        for i in range(x):
            
            s_= int(P[i][1])
            p = P[i][0]
            r = P[i][2]
            
            u += p*(r+gamma*V[s_])
            
        value += pi[s,a] * u
  
    V[s]=value
    return V[s]

In [28]:
def value_iteration(env, gamma, theta):
    V = np.zeros(env.observation_space.n)                                       # initialize v(0) to arbitory value, my case "zeros"
    while True:
        delta = 0
        for s in range(env.observation_space.n):                       # iterate for all states
            v = V[s]
            bellman_optimality_update(env, V, s, gamma)   # update state_value with bellman_optimality_update
            delta = max(delta, abs(v - V[s]))             # assign the change in value per iteration to delta  
        if delta < theta:                                       
            break                                         # if change gets to negligible 
                                                          # --> converged to optimal value         
    pi = np.zeros((env.observation_space.n, env.action_space.n)) 
    action = np.zeros((env.observation_space.n))
    for s in range(env.observation_space.n):
        pi = argmax(env, V, pi,action, s, gamma)         # extract optimal policy using action value 
        
    return V, pi,action                                          # optimal value funtion, optimal policy


In [29]:
V, pi, action = value_iteration(env, gamma, theta)
# tools.plot(V,pi)
# tools.plot(V, pi)

In [30]:
a= np.reshape(action,(8,8))
print(a)   

[[3. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 2. 2. 1.]
 [3. 3. 0. 0. 2. 3. 2. 1.]
 [3. 3. 3. 1. 0. 0. 2. 2.]
 [0. 3. 0. 0. 2. 1. 3. 2.]
 [0. 0. 0. 1. 3. 0. 0. 2.]
 [0. 0. 1. 0. 0. 0. 0. 2.]
 [0. 1. 0. 0. 1. 2. 1. 0.]]


In [31]:
print(action)

[3. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 2. 2. 1. 3. 3. 0. 0. 2. 3. 2. 1.
 3. 3. 3. 1. 0. 0. 2. 2. 0. 3. 0. 0. 2. 1. 3. 2. 0. 0. 0. 1. 3. 0. 0. 2.
 0. 0. 1. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 2. 1. 0.]


In [33]:
env = gym.make('FrozenLake8x8-v1')
e=0
for i_episode in range(100):
    observation, info = env.reset(seed=42)
    for t in range(80):
        # print(action[observation])
        observation, reward, terminated, truncated, info = env.step(int(action[observation]))
        if terminated or truncated:
            # print(t)
            if reward == 1:
                e +=1
            break
print(f"agent succeeded to reach goal {e} out of 100 Episodes using this policy ")
env.close()


agent succeeded to reach goal 100 out of 100 Episodes using this policy 
